In [1]:
import ee
import geemap

# Initialize the Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-raman')
# Define region of interest (ROI)
state = "rajasthan"
district = "bhilwara"
block = "mandalgarh"
block_ = "bhilwara_mandalgarh"

# List of assets
L1_asset = [
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2017-07-01_2018-06-30_LULCmap_10m'), 
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2018-07-01_2019-06-30_LULCmap_10m'), 
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2019-07-01_2020-06-30_LULCmap_10m'), 
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2020-07-01_2021-06-30_LULCmap_10m'), 
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2021-07-01_2022-06-30_LULCmap_10m'), 
    ee.Image('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/'+block_+'_2022-07-01_2023-06-30_LULCmap_10m')
]

# Filter for the region of interest
roi_boundary = ee.FeatureCollection('projects/df-project-iit/assets/core-stack/'+state+'/'+district+'/'+block+'/filtered_mws_'+block_+'_uid')

# Remap values function
def remap_values(image):
    return image.remap([1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12], 
                       [1, 2, 2, 2, 3, 4, 3, 3, 3, 3, 4], 
                       0, 'predicted_label')

L1_asset_remaped = [remap_values(asset) for asset in L1_asset]


# Create image collections
then = ee.ImageCollection(L1_asset_remaped[:3])
now = ee.ImageCollection(L1_asset_remaped[3:])

# Compute mode and clip
then = then.mode().clip(roi_boundary.geometry())
now = now.mode().clip(roi_boundary.geometry())

# Compute transitions
trans_bu_bu = then.eq(1).And(now.eq(1))
trans_w_bu = then.eq(2).And(now.eq(1)).multiply(2)
trans_tr_bu = then.eq(3).And(now.eq(1)).multiply(3)
trans_b_bu = then.eq(4).And(now.eq(1)).multiply(4)

# Create a zero image and add transitions
change_bu = ee.Image.constant(0).clip(roi_boundary.geometry())
change_bu = change_bu.add(trans_bu_bu).add(trans_w_bu).add(trans_tr_bu).add(trans_b_bu)

colors = ['#000000', '#00ff00', '#ff0000', '#ffff00', '#808080']
labels = ['Bg', 'Bu-Bu', 'Water-Bu', 'Tree/Farm-Bu', 'Barren/Shrub-Bu']

# Visualization parameters
VisParams = {
    'min': 0,
    'max': 4,
    'palette': colors
}

# Map visualization
Map = geemap.Map()
Map.addLayer(change_bu, VisParams, 'Change')
Map.centerObject(change_bu.geometry())

# Add the legend
legend = Map.add_legend(title="Classes", colors=colors, labels=labels)

# Display map
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
# Degradation
# Remap values function
def remap_values(image):
    return image.remap([1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12], 
                       [1, 2, 2, 2, 4, 5, 3, 3, 3, 3, 6], 
                       0, 'predicted_label')

L1_asset_remaped = [remap_values(asset) for asset in L1_asset]


# Create image collections
then = ee.ImageCollection(L1_asset_remaped[:3])
now = ee.ImageCollection(L1_asset_remaped[3:])

# Compute mode and clip
then = then.mode().clip(roi_boundary.geometry())
now = now.mode().clip(roi_boundary.geometry())

trans_f_f = then.eq(3).And(now.eq(3))
trans_f_bu = then.eq(3).And(now.eq(1)).multiply(2)
trans_f_Ba = then.eq(3).And(now.eq(5)).multiply(3)
trans_f_sc = then.eq(3).And(now.eq(6)).multiply(4)

# Create a zero image and add transitions
change_deg = ee.Image.constant(0).clip(roi_boundary.geometry())
change_deg = change_deg.add(trans_f_f).add(trans_f_bu).add(trans_f_Ba).add(trans_f_sc)

colors = ['#000000', '#00ff00', '#ff0000', '#ffff00', '#808080']
labels = ['Bg', 'Farm-Farm', 'Farm-Bu', 'Farm-Ba', 'Farm-Bu']

# Visualization parameters
VisParams = {
    'min': 0,
    'max': 4,
    'palette': colors
}

# Map visualization
Map = geemap.Map()
Map.addLayer(change_deg, VisParams, 'Change')
Map.centerObject(change_deg.geometry())

# Add the legend
legend = Map.add_legend(title="Classes", colors=colors, labels=labels)

# Display map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
# Deforestation
# Create an initial zero image
zero_image2 = ee.Image.constant(0).clip(L1_asset[0].geometry())

for i in range(1, 5):
    before = L1_asset[i-1]
    middle = L1_asset[i]
    after = L1_asset[i+1]

    cond1 = before.eq(12).And(after.eq(12)).And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))
    cond2 = before.eq(2).Or(before.eq(3)).Or(before.eq(4)).And(after.eq(2).Or(after.eq(3)).Or(after.eq(4))).And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))
    cond3 = before.eq(6).And(after.eq(6)).And(middle.eq(12))
    cond4 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)).And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))).And(middle.eq(12))
    cond5 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)).And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))).And(middle.eq(7))
    cond6 = before.eq(6).And(after.eq(6)).And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))
    cond7 = before.eq(8).Or(before.eq(9)).Or(before.eq(10)).Or(before.eq(11)).And(after.eq(8).Or(after.eq(9)).Or(after.eq(10)).Or(after.eq(11))).And(middle.eq(6))
    cond8 = before.eq(1).And(after.eq(1)).And(middle.eq(6))
    cond9 = before.eq(6).And(after.eq(6)).And(middle.eq(1))
    cond10 = before.eq(1).And(after.eq(1)).And(middle.eq(8).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))
    cond11 = before.eq(7).And(after.eq(7)).And(middle.eq(6).Or(middle.eq(8)).Or(middle.eq(9)).Or(middle.eq(10)).Or(middle.eq(11)))

    zero_image2 = zero_image2.add(cond1).add(cond2).add(cond3).add(cond4).add(cond5).add(cond6).add(cond7).add(cond8).add(cond9).add(cond10).add(cond11)
import copy
L1_asset_copy = copy.deepcopy(L1_asset)
for i in range(1, 5):
    before = L1_asset[i-1]
    middle = L1_asset[i]
    after = L1_asset[i+1]
    
    cond1 = before.eq(3).And(middle.neq(3)).And(after.eq(3)).And((zero_image2.eq(3).Or(zero_image2.eq(4))))
    cond2 = before.neq(3).And(middle.eq(3)).And(after.neq(3)).And((zero_image2.eq(3).Or(zero_image2.eq(4))))
    
    middle = middle.where(cond1, 3)
    middle = middle.where(cond2, before)
    
    L1_asset_copy[i] = middle

# Remap values function
def remap_values(image):
    remapped = image.remap(
        [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12], 
        [1, 2, 2, 2, 3, 5, 4, 4, 4, 4, 6], 
        0, 
        'predicted_label'
    )
    return remapped

L1_asset_remaped = [remap_values(asset) for asset in L1_asset_copy]


# Create image collections
then = ee.ImageCollection(L1_asset_remaped[:3])
now = ee.ImageCollection(L1_asset_remaped[3:])

# Compute mode and clip
then = then.mode().clip(roi_boundary.geometry())
now = now.mode().clip(roi_boundary.geometry())

trans_fo_fo = then.eq(3).And(now.eq(3))
trans_fo_bu = then.eq(3).And(now.eq(1)).multiply(2)
trans_fo_fa = then.eq(3).And(now.eq(4)).multiply(3)
trans_fo_ba = then.eq(3).And(now.eq(5)).multiply(4)
trans_sc = then.eq(3).And(now.eq(6)).multiply(5)

# Create a zero image and add transitions
change_def = ee.Image.constant(0).clip(roi_boundary.geometry())
change_def = change_def.add(trans_fo_fo).add(trans_fo_bu).add(trans_fo_fa).add(trans_fo_ba).add(trans_sc)

colors = ['#000000', '#00ff00', '#ff0000', '#ffff00', '#808080', '#909990']
labels = ['Bg', 'Forest-Forest', 'Forest-Builtup', 'Forest-Farm', 'Forest-Barren', 'Forest-Scrub']

# Visualization parameters
VisParams = {
    'min': 0,
    'max': 5,
    'palette': colors
}

# Map visualization
Map = geemap.Map()
Map.addLayer(change_def, VisParams, 'Change')
Map.centerObject(change_def.geometry())

# Add the legend
legend = Map.add_legend(title="Classes", colors=colors, labels=labels)

# Display map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
#afforestation

trans_fo_fo = then.eq(3).And(now.eq(3))
trans_bu_fo = then.eq(1).And(now.eq(3)).multiply(2)
trans_fa_fo = then.eq(4).And(now.eq(3)).multiply(3)
trans_ba_fo = then.eq(5).And(now.eq(3)).multiply(4)
trans_sc_fo = then.eq(6).And(now.eq(3)).multiply(5)

# Create a zero image and add transitions
change_af = ee.Image.constant(0).clip(roi_boundary.geometry())
change_af = change_af.add(trans_fo_fo).add(trans_bu_fo).add(trans_fa_fo).add(trans_ba_fo).add(trans_sc_fo)

colors = ['#000000', '#00ff00', '#ff0000', '#ffff00', '#808080', '#909990']
labels = ['Bg', 'Forest-Forest', 'Builtup-Forest', 'Farm-Forest', 'Barren-Forest', 'Scrub-Forest']

# Visualization parameters
VisParams = {
    'min': 0,
    'max': 5,
    'palette': colors
}

# Map visualization
Map = geemap.Map()
Map.addLayer(change_af, VisParams, 'Change')
Map.centerObject(change_af.geometry())

# Add the legend
legend = Map.add_legend(title="Classes", colors=colors, labels=labels)

# Display map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Cropping intensity
# Remap values function
def remap_values(image):
    return image.remap([1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12], 
                       [1, 2, 2, 2, 3, 4, 5, 5, 6,  7,  8], 
                       0, 'predicted_label')

L1_asset_remaped = [remap_values(asset) for asset in L1_asset]


# Create image collections
then = ee.ImageCollection(L1_asset_remaped[:3])
now = ee.ImageCollection(L1_asset_remaped[3:])

# Compute mode and clip
then = then.mode().clip(roi_boundary.geometry())
now = now.mode().clip(roi_boundary.geometry())

trans_do_si = then.eq(6).And(now.eq(5))
trans_tr_si = then.eq(7).And(now.eq(5)).multiply(2)
trans_tr_do = then.eq(7).And(now.eq(6)).multiply(3)
trans_si_do = then.eq(5).And(now.eq(6)).multiply(4)
trans_si_tr = then.eq(5).And(now.eq(7)).multiply(5)
trans_do_tr = then.eq(6).And(now.eq(7)).multiply(6)
trans_same = (then.eq(5).And(now.eq(5))).Or(then.eq(6).And(now.eq(6))).Or(then.eq(7).And(now.eq(7))).multiply(7)

# Create a zero image and add transitions
change_far = ee.Image.constant(0).clip(roi_boundary.geometry())
change_far = change_far.add(trans_do_si).add(trans_tr_si).add(trans_tr_do).add(trans_si_do).add(trans_si_tr).add(trans_do_tr).add(trans_same)

colors = ['#000000', '#00ff00', '#ff0000', '#ffff00', '#808080', '#025669', '#75151E', '#84c3be' ]
labels = ['Bg', 'Double-Single', 'Triple-Single', 'Triple-Double', 'Single-Double', 'Single-Triple', 'Double-Triple', 'As-it-is']

# Visualization parameters
VisParams = {
    'min': 0,
    'max': 4,
    'palette': colors
}

palette = [
    '000000', '000000', '000000', '000000', '000000', 'f75cff', 
    '991695', 'e6ab2c', '3bef34', '2baa25', 'e6ef34', 'a1a524', 'eea15e'
]


vis_params = {
    'min': 0,
    'max': 12,
    'palette': palette
}
# Map visualization
Map = geemap.Map()
url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
Map.layout.height = '1000px'
Map.add_tile_layer(url, name="Google Map", attribution="Google")
Map.addLayer(change_far, VisParams, 'Change')

Map.addLayer(L1_asset[0], vis_params, 'LULC')
#Map.addLayer(L1_asset[0], )
Map.centerObject(change_far.geometry())

# Add the legend
legend = Map.add_legend(title="Classes", colors=colors, labels=labels)

# Display map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
change_asset = [change_bu, change_deg, change_def, change_af, change_far][4:]

In [7]:
change_asset

In [8]:
for index, change in enumerate(["Urbanization", "Degradation", "Deforestation", "Afforestation", "CropIntensity"][4:]):
    task = ee.batch.Export.image.toAsset(
        image=change_asset[index],
        description= block_+'_' + change,
        assetId='projects/ee-raman/assets/change/'+block_+"_"+change,
        pyramidingPolicy = {'predicted_label': 'mode'},
        scale = 10,
        maxPixels = 1e13,
        crs = 'EPSG:4326'
    )
    task.start()
